In [1]:
inp = "1999/10/19"

In [2]:
import re
datefmt = re.compile(r"(\d{4})\/(\d{2})\/(\d{2})")

if datefmt.match(inp):
    print("ok")

ok


In [3]:
import logging
import re
import requests

class NYTGrid:
    
    BASE_URL = "https://raw.githubusercontent.com/doshea/nyt_crosswords/master/"
    
    def __init__(self, date):
        datefmt = re.compile(r"(\d{4})\/(\d{2})\/(\d{2})")
        if not datefmt.match(inp):
            raise ValueError("date must be in the format YYYY/MM/DD")
        ymd = date.split("/")
        self._year = ymd[0]
        self._month = ymd[1]
        self._day = ymd[2]
        self._raw_data = None
        self.title = None
        self.author = None
        
        
        
    def fetch_data(self):
        target = f"{self.BASE_URL}{self._year}/{self._month}/{self._day}.json"
        r = requests.get(target)
        r.raise_for_status()
        self._raw_data = r.json()
        self.title = self._raw_data["title"]
        self.author = self._raw_data["author"]
        self.size = self._raw_data["size"]
        
    def __len__(self):
        return self.size["cols"]
    
    def __str__(self):
        out = "{:^50}".format(self.title)
        out += "\n"
        for i in range(self.size["rows"]):
            start = self.size["cols"]*(i)
            end = self.size["cols"]*(i+1)
            out += "\n"
            out+= (" | ".join(self._raw_data["grid"][start:end]).replace(".", " "))
        return out 

In [4]:
g = NYTGrid("1997/09/28")

In [5]:
g.fetch_data()

In [6]:
print(g)

X MARKS THE SPOT                 

A | G | H | A | S |   | F | R | E | D |   | S | A | I | D |   | D | E | F | E | R
L | E | A | N | T |   | L | O | N | I |   | P | A | T | E |   | A | Q | A | B | A
I | N | T | E | R | R | U | P | T | S |   | I | N | S | C |   | B | U | R | B | S
B | E | T | W | E | E | N | Y | O | U | A | N | D | M | E |   | S | I | F | T | S
I | R | E |   | A | L | G |   |   | S | L | O | W | E | D |   | A | P | R | I | L
S | A | D |   | M | A | A |   | S | I | T | U |   |   | E | A | T |   | O | D | E
  |   |   | N | E | X | T | T | O | N | O | T | H | I | N | G |   | A | M | E | S
T | U | B | E | R |   |   | R | A | G |   |   | A | N | T | E | S | U | P |   |  
U | T | E | S |   | C | H | I | P |   | A | B | I | E |   |   | O | R | E | S |  
B | O | Y | S |   | A | U | K |   | S | A | R | K |   | C | A | B | A | R | E | T
A | P | O |   | A | F | T | E | R | T | H | O | U | G | H | T | S |   | F | D | A
S | I | N | G | L | E | T |   | A | Y | E | S |   | L | E | O |

In [7]:
dta = g._raw_data
dta.keys()

dict_keys(['acrossmap', 'admin', 'answers', 'author', 'autowrap', 'bbars', 'circles', 'clues', 'code', 'copyright', 'date', 'dow', 'downmap', 'editor', 'grid', 'gridnums', 'hold', 'id', 'id2', 'interpretcolors', 'jnotes', 'key', 'mini', 'notepad', 'publisher', 'rbars', 'shadecircles', 'size', 'title', 'track', 'type'])